In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from datetime import datetime,timedelta

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={
    
    'Vendas':
    
    """
    
    SELECT * FROM netfeira.vw_targetestatistico
    
    """,
    
    'Estoque':
    
    """
    
    SELECT * FROM netfeira.vw_estoque
    WHERE Tipo='CENTRAL'    
    
    """
    
}

In [3]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [82]:
vendas_df=pd.DataFrame()

vendas_df=tabelas_df['Vendas']

In [83]:
vendas_df.columns

Index(['Origem', 'Situação Entrega', 'ID Situação', 'Situação', 'Pedido',
       'NFe', 'Tipo de Pedido', 'Tipo de Operação', 'Tabelas',
       'Data de Emissão', 'Data de Faturamento', 'Data de Entrega',
       'ID Empresa', 'ID Cliente', 'ID Vendedor', 'SKU', 'Seq', 'Qtde',
       'Unid. VDA', 'Fator', 'Qtde. VDA', 'Valor VDA', 'Total Venda',
       'Total AV', 'MG CRP', 'Margem CUE', 'Margem CMP', 'Comsissão R$',
       'COFINS R$', 'PIS R$', 'ICMS R$', 'ICMS ST R$', 'IPI R$',
       'Peso Bruto KG', 'Peso Líquido KG', 'Total Geral'],
      dtype='object')

In [84]:
vendas_df=vendas_df.loc[(vendas_df['Tipo de Operação']!='OUTROS')&(vendas_df['ID Situação'].isin(['FA','AB']))]

In [85]:
data_inicio=datetime.now()-timedelta(days=30)

data_atual=datetime.now()

In [86]:
vendas_df=vendas_df.loc[vendas_df['Data de Emissão'].between(data_inicio,data_atual)]

In [87]:
vendas_df=vendas_df[['SKU','Qtde']].groupby(['SKU'],as_index=False).mean()

In [88]:
estoque_df=pd.DataFrame()

estoque_df=tabelas_df['Estoque']

In [89]:
estoque_df=estoque_df.merge(vendas_df,on='SKU',how='left')

In [90]:
estoque_df=estoque_df.loc[estoque_df['Qtde Disponível']>0]

In [91]:
estoque_df

,Local,Tipo,SKU,Cód. Fabricante,Produto,Fabricante,Departamento,Seção,Categoria,Linha,Fator CX,Unid. CMP,Estoque,Qtde Pendente,Qtde Disponível,Caixa Disponível,Peso Disponível,Custo C/ST,Custo Total,Qtde CMP,Última Venda,Qtde
0,DEMARCHI CENTRAL,CENTRAL,3,101111036,POLPINHA DE ABACAXI (100G),DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,6.0,KG,483,16,467,77.0,467.0000,13.5406,6323.46,467.0000,2022-09-27,3.531401
2,DEMARCHI CENTRAL,CENTRAL,12,101110206,POLPINHA DE ACEROLA (100G),DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,6.0,KG,147,32,115,19.0,115.0000,12.7966,1471.61,115.0000,2022-09-27,5.400000
3,DEMARCHI CENTRAL,CENTRAL,34,101122213,"IQF AMORA CONGELADOS (1,200 KG)",DE MARCHI,CONGELADOS,FRUTAS CONGELADAS,FRUTAS > 1KG,"FRUTA 1,2KG",9.0,KG,942,13,929,103.0,1115.8824,37.6815,35006.11,1115.8824,2022-09-27,4.417910
4,DEMARCHI CENTRAL,CENTRAL,35,101112306,POLPINHA DE AMORA (100G),DE MARCHI,CONGELADOS,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,6.0,KG,144,0,144,24.0,144.0000,50.4425,7263.72,144.0000,2022-08-11,NaN
5,DEMARCHI CENTRAL,CENTRAL,47,101136573,"CONGELADO ERVILHA DM (1,2KG)",DE MARCHI,CONGELADOS,VEGETAIS CONGELADOS,ERVILHA,"LEGUMES ATÉ 2,5KG",8.0,KG,539,3,536,67.0,644.3060,14.2800,7654.08,644.3060,2022-09-26,7.055556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,DEMARCHI CENTRAL,CENTRAL,9592,1000010284,CONG. BATATA MCCAIN AIRFRYER EXTRA CROC 600G (...,MCCAIN,CONGELADOS,BATATAS CONGELADAS,BATATA VAREJO,BATATA PALITO,16.0,CX,3514,0,3514,219.0,2108.4000,9.9911,35108.73,219.0000,2022-09-26,13.820000
463,DEMARCHI CENTRAL,CENTRAL,9593,101132853,"CONGELADO BETERRABA CUBO (1,02KG)",DE MARCHI,CONGELADOS,VEGETAIS CONGELADOS,BETERRABA,"LEGUMES ATÉ 2,5KG",0.0,UN,41,0,41,NaN,42.8369,6.7000,274.70,41.0000,NaT,NaN
464,DEMARCHI CENTRAL,CENTRAL,9594,504011,BROWNIE CHOCOLATE UNB 1500G UNB,BIMBO,CONGELADOS,PADARIA,BROWNIE,BROWNIE,30.0,CX,333,0,333,11.0,16.6500,2.5868,861.40,11.0000,2022-09-26,3.725000
465,DEMARCHI CENTRAL,CENTRAL,9595,1000009716,"CONG. BATATA MCCAIN CORTE FINO (5X2,5 KG)",MCCAIN,CONGELADOS,BATATAS CONGELADAS,BATATA CORTE FINO,BATATA 7MM,5.0,CX,1750,0,1750,350.0,4375.0000,29.3120,51296.00,350.0000,NaT,NaN
